In [ ]:
'''!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json'''


In [ ]:
'''!kaggle datasets download -d vitaminc/cigarette-smoker-detection
!unzip cigarette-smoker-detection.zip -d /content/smoking_dataset'''


Streaming output truncated to the last 5000 lines.
  inflating: /content/smoking_dataset/data/data/smoking/000335.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000337.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000338.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000339.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000343.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000344.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000352.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000353.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000354.png  
  inflating: /content/smoking_dataset/data/data/smoking/000356.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000357.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000359.jpg  
  inflating: /content/smoking_dataset/data/data/smoking/000362.jpg  
  inflating: /content/smoking_dataset/data/data/smok

In [3]:
'''now We will:

Split 70% → train

Split 15% → val

Split 15% → test

Preserve class balance

Use random but reproducible split'''


import os
import random
import shutil
from tqdm import tqdm

random.seed(42)

BASE_DIR = "/content/smoking_dataset"
RAW_DIR = os.path.join(BASE_DIR, "data")

SPLIT_DIRS = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}

classes = ["smoking", "not_smoking"]

# Create folder structure
for split in SPLIT_DIRS:
    for cls in classes:
        os.makedirs(os.path.join(BASE_DIR, split, cls), exist_ok=True)

# Split data
for cls in classes:
    cls_path = os.path.join(RAW_DIR, cls)
    images = os.listdir(cls_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(SPLIT_DIRS["train"] * total)
    val_end = train_end + int(SPLIT_DIRS["val"] * total)

    splits = {
        "train": images[:train_end],
        "val": images[train_end:val_end],
        "test": images[val_end:]
    }

    for split, split_images in splits.items():
        for img in tqdm(split_images, desc=f"{cls} → {split}"):
            src = os.path.join(cls_path, img)
            dst = os.path.join(BASE_DIR, split, cls, img)
            shutil.copy(src, dst)


not_smoking → test: 100%|██████████| 193/193 [00:05<00:00, 32.28it/s]


In [4]:
# Verify the Split

def count_images(path):
    return len(os.listdir(path))

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    for cls in classes:
        print(cls, ":", count_images(f"{BASE_DIR}/{split}/{cls}"))



TRAIN
smoking : 1397
not_smoking : 895

VAL
smoking : 299
not_smoking : 191

TEST
smoking : 300
not_smoking : 193


In [5]:
# DataLoader & Transforms

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

DATA_DIR = "/content/smoking_dataset"

In [6]:
# Train Transforms (With Augmentation)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Validation & Test Transforms (NO Augmentation)
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [7]:
# Create Datasets

train_dataset = datasets.ImageFolder(
    root=os.path.join(DATA_DIR, "train"),
    transform=train_transforms
)

val_dataset = datasets.ImageFolder(
    root=os.path.join(DATA_DIR, "val"),
    transform=val_test_transforms
)

test_dataset = datasets.ImageFolder(
    root=os.path.join(DATA_DIR, "test"),
    transform=val_test_transforms
)


In [8]:
# Create DataLoaders

BATCH_SIZE = 32
NUM_WORKERS = 2  # Colab safe

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


In [9]:
# Verify Everything

print("Classes:", train_dataset.classes)
print("Class to Index:", train_dataset.class_to_idx)

images, labels = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)


Classes: ['not_smoking', 'smoking']
Class to Index: {'not_smoking': 0, 'smoking': 1}
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])


In [10]:
# MobileNetV2 Training

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [11]:
#Load Pretrained MobileNetV2

model = models.mobilenet_v2(pretrained=True)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 136MB/s]


In [12]:
# Freeze Backbone
'''Why?

Prevents overfitting

Trains faster

Uses general visual features'''


for param in model.parameters():
    param.requires_grad = False


In [13]:
# Replace Classifier Head

''' MobileNetV2 default classifier: (model.classifier)
Replace it with '''

num_features = model.classifier[1].in_features

model.classifier = nn.Sequential(
    nn.Dropout(p=0.3),
    nn.Linear(num_features, 2)  # smoking / not_smoking
)

model = model.to(device)


In [14]:
# Loss Function & Optimizer

criterion = nn.CrossEntropyLoss()

# Optimizer (Classifier Only)

optimizer = optim.Adam(
    model.classifier.parameters(),
    lr=1e-3
)


In [15]:
# Training Loop

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    return running_loss / len(loader), acc


In [16]:
# Validation Loop

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    return running_loss / len(loader), acc


In [17]:
# Full Training Loop

EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion
    )

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


Epoch [1/10] | Train Acc: 0.7173 | Val Acc: 0.7762
Epoch [2/10] | Train Acc: 0.8051 | Val Acc: 0.7885
Epoch [3/10] | Train Acc: 0.8143 | Val Acc: 0.7947
Epoch [4/10] | Train Acc: 0.8165 | Val Acc: 0.7803
Epoch [5/10] | Train Acc: 0.8060 | Val Acc: 0.7967
Epoch [6/10] | Train Acc: 0.8235 | Val Acc: 0.7721
Epoch [7/10] | Train Acc: 0.8284 | Val Acc: 0.7947
Epoch [8/10] | Train Acc: 0.8292 | Val Acc: 0.8111
Epoch [9/10] | Train Acc: 0.8358 | Val Acc: 0.7926
Epoch [10/10] | Train Acc: 0.8248 | Val Acc: 0.7721


In [18]:
# Save Model
torch.save(model.state_dict(),"smoking_mobilenetv2.pth")


In [19]:
# Evaluation & Inference

# Load Trained Model

from torchvision import models
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(pretrained=False)

num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, 2)
)

model.load_state_dict(torch.load("smoking_mobilenetv2.pth", map_location=device))
model = model.to(device)
model.eval()


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [20]:
# Test Set Evaluation

def evaluate(model, loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [21]:
test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc:.4f}")


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Test Accuracy: 0.7832


In [22]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())


print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds,
      target_names=["Not Smoking", "Smoking"]))


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[[151  42]
 [ 64 232]]
              precision    recall  f1-score   support

 Not Smoking       0.70      0.78      0.74       193
     Smoking       0.85      0.78      0.81       296

    accuracy                           0.78       489
   macro avg       0.77      0.78      0.78       489
weighted avg       0.79      0.78      0.78       489



In [23]:
# Single Image Inference

from PIL import Image
from torchvision import transforms

infer_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [24]:
# Prediction Function

def predict_smoking(image_path):
    image = Image.open(image_path).convert("RGB")
    image = infer_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        probs = torch.softmax(output, dim=1)
        confidence, pred = torch.max(probs, 1)

    label_map = {0: "Not Smoking", 1: "Smoking"}

    return {
        "prediction": label_map[pred.item()],
        "confidence": round(confidence.item(), 3)
    }


In [26]:
predict_smoking("/content/smoking_dataset/test/smoking/000389.jpeg")


{'prediction': 'Smoking', 'confidence': 0.918}

In [29]:
predict_smoking("/content/smoking_dataset/test/not_smoking/ggg553.jpg")


{'prediction': 'Smoking', 'confidence': 0.968}